In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path, PurePath
import requests
from requests.exceptions import HTTPError, ConnectionError
from ipywidgets import interact
import ipywidgets as widgets
from rank_bm25 import BM25Okapi
import nltk
from nltk.corpus import stopwords
from src.kagle import XmlsParse
nltk.download("punkt")
import re
import os
import json
import glob
import sys
from tqdm import tqdm
from rank_bm25 import BM25Okapi

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
parser = XmlsParse("C:/Users/guill/Documents/fac/master/2A/RI/recherche-info/src/data/coll/3951433.xml")
parser2 = XmlsParse("C:/Users/guill/Documents/fac/master/2A/RI/recherche-info/src/data/coll/22478.xml")

parser.parse()
parser2.parse()

s = [parser.corpusWcount['3951433'], parser2.corpusWcount['22478']]


[['list', 'statutory', 'instruments', 'united', 'kingdom,', '1992', '3951433', '240437342', '2008-09-23t13:40:09z', 'cornellrockey', '266817', '1992', 'united', 'kingdom', 'lists', 'statutory', 'instruments', 'united', 'kingdom', 'ambox-style"', 'style=""', 'kb', 'this', 'article', '100', 'size.', 'some', 'browsers', 'may', 'difficulty', 'rendering', 'article.', 'please', 'consider', '.', 'uk', 'legislation', 'acts', 'parliament', 'states', 'preceding', 'united', 'kingdom', 'kingdom', 'england', 'before', '1601', '1603–1641', 'interregnum', '(1642–1660)', '1660–1699', '1700–1706', 'of', '·', '·', 'kingdom', 'ireland', 'before', '1701', '1701–1800', 'of', '·', 'kingdom', 'scotland', 'up', '1707', 'of', 'kingdom', 'great', 'britain', '1707–1719', '1720–1739', '1740–1759', '1760–1779', '1780–1800', 'of', '·', '·', '·', '·', 'acts', 'united', 'kingdom', 'parliament', '1801–1819', '1820–1839', '1840–1859', '1860–1879', '1880–1899', '1900–1919', '1920–1939', '1940–1959', '1960–1979', '1980–1

In [3]:
english_stopwords = list(set(stopwords.words('english')))

def strip_characters(text):
    t = re.sub('\(|\)|:|,|;|\.|’|”|“|\?|%|>|<', '', text)
    t = re.sub('/', ' ', t)
    t = t.replace("'",'')
    return t

def clean(text):
    t = text.lower()
    t = strip_characters(t)
    return t

def tokenize(text):
    words = nltk.word_tokenize(text)
    return list(set([word for word in words 
                     if len(word) > 1
                     and not word in english_stopwords
                     and not (word.isnumeric() and len(word) is not 4)
                     and (not word.isnumeric() or word.isalpha())] )
               )

def preprocess(text):
    t = clean(text)
    tokens = tokenize(t)
    return tokens

class SearchResults:
    
    def __init__(self, 
                 data: pd.DataFrame,
                 columns = None):
        self.results = data
        if columns:
            self.results = self.results[columns]
            
    
    def __len__(self):
        return len(self.results)
        
    def _repr_html_(self):
        return self.results._repr_html_()

SEARCH_DISPLAY_COLUMNS = ['_id', 'title', 'abstract', 'text']
class WordTokenIndex:
    
    def __init__(self, 
                 corpus: pd.DataFrame, 
                 columns=SEARCH_DISPLAY_COLUMNS):
        self.corpus = corpus
        raw_search_str = self.corpus.abstract.fillna('') + ' ' + self.corpus.title.fillna('') + ' ' + self.corpus.text.fillna('')
        self.index = raw_search_str.apply(preprocess).to_frame()
        self.index.columns = ['terms']
        self.index.index = self.corpus.index
        self.columns = columns
    
    def search(self, search_string):
        search_terms = preprocess(search_string)
        result_index = self.index.terms.apply(lambda terms: any(i in terms for i in search_terms))
        results = self.corpus[result_index].copy().reset_index().rename(columns={'index':'paper'})
        return SearchResults(results, self.columns + ['paper'])
class RankBM25Index(WordTokenIndex):
    
    def __init__(self, corpus: pd.DataFrame, columns=SEARCH_DISPLAY_COLUMNS):
        super().__init__(corpus, columns)
        self.bm25 = BM25Okapi(self.index.terms.tolist())
        
    def search(self, search_string, n=4):
        search_terms = preprocess(search_string)
        doc_scores = self.bm25.get_scores(search_terms)
        ind = np.argsort(doc_scores)[::-1][:n]
        results = self.corpus.iloc[ind][self.columns]
        results['Score'] = doc_scores[ind]
        results['orig_ind'] = ind
        results = results[results.Score > 0]
        return SearchResults(results.reset_index(), self.columns + ['Score', 'orig_ind'])
bm25_index = RankBM25Index(s)

AttributeError: 'list' object has no attribute 'abstract'